# Covid-19 & Blood Type

<i>Version 1</i>

## References

* <i><b>"Blood Type May Affect COVID-19 Risk: Study"</b></i> [WebMD](https://www.webmd.com/lung/news/20200320/blood-type-may-affect-covid19-risk-study)
* <i><b>"Relationship between the ABO Blood Group and the COVID-19 Susceptibility"</b></i> by Jiao Zhao, et al. [Abstract & link to PDF](https://www.medrxiv.org/content/10.1101/2020.03.11.20031096v1)
* <i><b>"Blood type distribution by country"</b></i> [Wikipedia](https://en.wikipedia.org/wiki/Blood_type_distribution_by_country)

## Module Imports

In [1]:
#import scipy
#from scipy.stats import multinomial
import numpy as np

## Raw Data

<u><b>Excerpt from Zhao, et al:</b></u> <i>"The ABO group in 3694 normal people in Wuhan showed a distribution of 32.16%,
24.90%, 9.10% and 33.84% for A, B, AB and O, respectively, versus the distribution
of 37.75%, 26.42%, 10.03% and 25.80% for A, B, AB and O, respectively, in 1,775
COVID-19 patients from Wuhan Jinyintan Hospital. The proportion of blood group A
and O in COVID-19 patients were significantly higher and lower, respectively, than
    that in normal people (both P < 0.001)."</i>

In [2]:
# From Table 1

# Controls (Wuhan Area), 3694 total:
wuhan_controls = np.array([1188, 920, 336, 1250])

# Wuhan Jinyintan Hospital
# Patients, 1775 total:
wuhan_patients = np.array([670, 469, 178, 458])
# Deaths, 206 total:
wuhan_deaths = np.array([85, 50, 19, 52])

# Renmin Hospital of Wuhan University
# patients, 113 total:
renmin_patients = np.array([45, 25, 15, 28])

# Controls (Shenzhen area), 23386 total:
shenzhen_controls = np.array([6728, 5880, 1712, 9066])

# Patients from Shenzhen Third People's Hospital, 285 total:
shenzhen_patients = np.array([82, 83, 39, 81])

In [11]:
# Wuhan Controls Distribution
print(f"Wuhan Controls Total: {wuhan_controls.sum()}")

wuhan_controls_dist = wuhan_controls / wuhan_controls.sum()
print(f"Wuhan Constrols Dist:\n  [A, B, AB, O] = {wuhan_controls_dist}")

Wuhan Controls Total: 3694
Wuhan Constrols Dist:
  [A, B, AB, O] = [0.3216026  0.24905252 0.09095831 0.33838657]


In [10]:
sick_or_dead = wuhan_patients + wuhan_deaths + renmin_patients + shenzhen_patients
print(sick_or_dead)
print(f"Totoal = {sick_or_dead.sum()}")

[882 627 251 619]
Totoal = 2379


In [15]:
foo = np.random.multinomial(wuhan_controls.sum(), wuhan_controls_dist, size=10)
print(foo)

[[1185  912  343 1254]
 [1230  888  325 1251]
 [1191  908  329 1266]
 [1184  941  339 1230]
 [1172  947  331 1244]
 [1204  911  338 1241]
 [1218  966  317 1193]
 [1232  889  297 1276]
 [1180  928  354 1232]
 [1197  924  343 1230]]


In [17]:
for f in foo:
    print(f / f.sum())

[0.32079047 0.24688684 0.09285328 0.33946941]
[0.33297239 0.24038982 0.08798051 0.33865728]
[0.32241473 0.24580401 0.08906335 0.34271792]
[0.32051976 0.25473741 0.09177044 0.33297239]
[0.31727125 0.25636167 0.08960476 0.33676232]
[0.32593395 0.24661613 0.09149973 0.33595019]
[0.32972388 0.26150514 0.08581483 0.32295615]
[0.33351381 0.24066053 0.08040065 0.34542501]
[0.31943692 0.25121819 0.09583108 0.33351381]
[0.32403898 0.25013535 0.09285328 0.33297239]


In [19]:
import pymc3 as pm

ModuleNotFoundError: No module named 'pymc3'

See https://discourse.pymc.io/t/how-to-improve-a-multinomial-model-from-bda3/2188

In [ ]:
with pm.Model() as m:
    ...:     theta = pm.Dirichlet('theta', a=np.ones_like(obs))
    ...:     post = pm.Multinomial('post', n=obs.sum(), p=theta, observed=obs)
    ...:     trace = pm.sample(1000, tune=5000)

In [20]:
np.ones_like(sick_or_dead)

array([1, 1, 1, 1])